### A cluster has been created for this demo
To run this demo, just select the cluster `dbdemos-llm-rag-chatbot-ling_huang` from the dropdown menu ([open cluster configuration](https://dbc-57a6b8b4-32b6.cloud.databricks.com/#setting/clusters/1212-021800-o0bqrfse/configuration)). <br />
*Note: If the cluster was deleted after 30 days, you can re-create it with `dbdemos.create_cluster('llm-rag-chatbot')` or re-install the demo: `dbdemos.install('llm-rag-chatbot')`*

# 2/ Creating the chatbot with Retrieval Augmented Generation (RAG)

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-self-managed-flow-2.png?raw=true" style="float: right; margin-left: 10px"  width="900px;">

Our Vector Search Index is now ready!

Let's now create and deploy a new Model Serving Endpoint to perform RAG.

The flow will be the following:

- A user asks a question
- The question is sent to our serverless Chatbot RAG endpoint
- The endpoint compute the embeddings and searches for docs similar to the question, leveraging the Vector Search Index
- The endpoint creates a prompt enriched with the doc
- The prompt is sent to the Foundation Model Serving Endpoint
- We display the output to our users!


<!-- Collect usage data (view). Remove it to disable collection or disable tracker during installation. View README for more details.  -->
<img width="1px" src="https://ppxrzfxige.execute-api.us-west-2.amazonaws.com/v1/analytics?category=data-science&org_id=local&notebook=%2F02-Deploy-RAG-Chatbot-Model&demo_name=llm-rag-chatbot&event=VIEW&path=%2F_dbdemos%2Fdata-science%2Fllm-rag-chatbot%2F02-Deploy-RAG-Chatbot-Model&version=1">

*Note: RAG performs document searches using Databricks Vector Search. In this notebook, we assume that the search index is ready for use. Make sure you run the previous [01-Data-Preparation-and-Index]($./01-Data-Preparation-and-Index [DO NOT EDIT]) notebook.*


In [0]:
%pip install "git+https://github.com/mlflow/mlflow.git@gateway-migration" langchain==0.0.344 databricks-vectorsearch==0.22 databricks-sdk==0.12.0 mlflow[databricks]
dbutils.library.restartPython()

In [0]:
%run ./_resources/00-init $reset_all_data=false

  
###  This demo requires a secret to work:
Your Model Serving Endpoint needs a secret to authenticate against your Vector Search Index (see [Documentation](https://docs.databricks.com/en/security/secrets/secrets.html)).  <br/>
- You'll need to setup the Databricks CLI on your laptop or using this cluster terminal: <br/>
`pip install databricks-cli` <br/>
- Configure the CLI. You'll need your workspace URL and a PAT token from your profile page<br>
`databricks configure`
- Create the dbdemos scope:<br/>
`databricks secrets create-scope dbdemos`
- Save your service principal secret. It will be used by the Model Endpoint to autenticate. If this is a demo/test, you can use one of your PAT token.<br>
`databricks secrets put-secret dbdemos rag_sp_token`

*Note: Make sure your service principal has access to the Vector Search index:*

```
spark.sql('GRANT USAGE ON CATALOG <catalog> TO `<YOUR_SP>`');
spark.sql('GRANT USAGE ON DATABASE <catalog>.<db> TO `<YOUR_SP>`');
from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c
WorkspaceClient().grants.update(c.SecurableType.TABLE, <index_name>, 
                                changes=[c.PermissionsChange(add=[c.Privilege["SELECT"]], principal="<YOUR_SP>")])
  ```

In [0]:
sp_name = spark.sql('select current_user() as user').collect()[0]['user'] 

In [0]:
print(sp_name)

In [0]:
catalog = "ling_test_demo"
db = "default"
index_name="ling_test_demo.default.databricks_documentation_vs_index"

# Make sure you replace sp_name with the SP owning the token in the secret. It has be the principal in the PAT token used in the model
sp_name = spark.sql('select current_user() as user').collect()[0]['user'] #Set to 

In [0]:
spark.sql(f'GRANT USAGE ON CATALOG {catalog} TO `{sp_name}`');
spark.sql(f'GRANT USAGE ON DATABASE {catalog}.{db} TO `{sp_name}`');

In [0]:

from databricks.sdk import WorkspaceClient
import databricks.sdk.service.catalog as c
WorkspaceClient().grants.update(c.SecurableType.TABLE, index_name, 
                                changes=[c.PermissionsChange(add=[c.Privilege["SELECT"]], principal=sp_name)])

### Langchain retriever

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-self-managed-model-1.png?raw=true" style="float: right" width="500px">

Let's start by building our Langchain retriever. 

It will be in charge of:

* Creating the input question embeddings (with Databricks `bge-large-en`)
* Calling the vector search index to find similar documents to augment the prompt with

Databricks Langchain wrapper makes it easy to do in one step, handling all the underlying logic and API call for you.

In [0]:
# url used to send the request to your model from the serverless endpoint
host = "https://" + spark.conf.get("spark.databricks.workspaceUrl")
print(host)

# os.environ['DATABRICKS_TOKEN'] = dbutils.secrets.get("ling_test_demo", "default")
os.environ['DATABRICKS_TOKEN'] = '-'

In [0]:
from databricks.vector_search.client import VectorSearchClient
from langchain.vectorstores import DatabricksVectorSearch
from langchain.embeddings import DatabricksEmbeddings

# Test embedding Langchain model
#NOTE: your question embedding model must match the one used in the chunk in the previous model 
embedding_model = DatabricksEmbeddings(endpoint="databricks-bge-large-en")
print(f"Test embeddings: {embedding_model.embed_query('What is Apache Spark?')[:20]}...")

def get_retriever(persist_dir: str = None):
    os.environ["DATABRICKS_HOST"] = host
    #Get the vector search index
    vsc = VectorSearchClient(workspace_url=host, personal_access_token=os.environ["DATABRICKS_TOKEN"])
    vs_index = vsc.get_index(
        endpoint_name=VECTOR_SEARCH_ENDPOINT_NAME,
        index_name=index_name
    )

    # Create the retriever
    vectorstore = DatabricksVectorSearch(
        vs_index, text_column="content", embedding=embedding_model
    )
    return vectorstore.as_retriever()


# test our retriever
vectorstore = get_retriever()
similar_documents = vectorstore.get_relevant_documents("How do I track my Databricks Billing?")
print(f"Relevant documents: {similar_documents[0]}")

### Building Databricks Chat Model to query llama-2-70b-chat foundation model

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-self-managed-model-3.png?raw=true" style="float: right" width="500px">

Our chatbot will be using llama2 foundation model to provide answer. 

While the model is available using the built-in [Foundation endpoint](/ml/endpoints) (using the `/serving-endpoints/databricks-llama-2-70b-chat/invocations` API), we can use Databricks Langchain Chat Model wrapper to easily build our chain.  

Note: multipe type of endpoint or langchain models can be used:

- Databricks Foundation models (what we'll use)
- Your fined-tune model
- An external model provider (such as Azure OpenAI)

In [0]:
# Test Databricks Foundation LLM model
from langchain.chat_models import ChatDatabricks
chat_model = ChatDatabricks(endpoint="databricks-llama-2-70b-chat", max_tokens = 200)
print(f"Test chat model: {chat_model.predict('What is Apache Spark')}")


### Assembling the complete RAG Chain

<img src="https://github.com/databricks-demos/dbdemos-resources/blob/main/images/product/chatbot-rag/llm-rag-self-managed-model-2.png?raw=true" style="float: right" width="600px">


Let's now merge the retriever and the model in a single Langchain chain.

We will use a custom langchain template for our assistant to give proper answer.

Make sure you take some time to try different templates and adjust your assistant tone and personality for your requirement.




In [0]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain.chat_models import ChatDatabricks

TEMPLATE = """You are an assistant for Databricks users. You are answering python, coding, SQL, data engineering, spark, data science, DW and platform, API or infrastructure administration question related to Databricks. If the question is not related to one of these topics, kindly decline to answer. If you don't know the answer, just say that you don't know, don't try to make up an answer. Keep the answer as concise as possible.
Use the following pieces of context to answer the question at the end:
{context}
Question: {question}
Answer:
"""
prompt = PromptTemplate(template=TEMPLATE, input_variables=["context", "question"])

chain = RetrievalQA.from_chain_type(
    llm=chat_model,
    chain_type="stuff",
    retriever=get_retriever(),
    chain_type_kwargs={"prompt": prompt}
)

In [0]:
# langchain.debug = True #uncomment to see the chain details and the full prompt being sent
question = {"query": "How can I track billing usage on my workspaces?"}
answer = chain.run(question)
print(answer)

### Saving our model to Unity Catalog registry

Now that our model is ready, we can register it within our Unity Catalog schema:

In [0]:
#dbdemos__delete_this_cell
#force the experiment to the field demos one. Required to launch as a batch
init_experiment_for_batch("chatbot-rag-llm", "simple")

In [0]:
from mlflow.models import infer_signature
import mlflow

mlflow.set_registry_uri("databricks-uc")
model_name = f"{catalog}.{db}.dbdemos_chatbot_model"

with mlflow.start_run(run_name="dbdemos_chatbot_rag") as run:
    signature = infer_signature(question, answer)
    model_info = mlflow.langchain.log_model(
        chain,
        loader_fn=get_retriever,  # Load the retriever with DATABRICKS_TOKEN env as secret (for authentication).
        artifact_path="chain",
        registered_model_name=model_name,
        pip_requirements=[
            "git+https://github.com/mlflow/mlflow.git@gateway-migration",
            "langchain==" + langchain.__version__,
            "databricks-vectorsearch",
        ],
        input_example=question,
        signature=signature
    )
    #------------------------
    # TODO: temporary fix to add the wheel, we won't need this after we switch to using PyPI
    import mlflow.models.utils
    mlflow.models.utils.add_libraries_to_model(
        f"models:/{model_name}/{get_latest_model_version(model_name)}"
    )

### Deploying our Chat Model as a Serverless Model Endpoint 

Our model is saved in Unity Catalog. The last step is to deploy it as a Model Serving.

We'll then be able to sending requests from our assistant frontend.

In [0]:
# Create or update serving endpoint
from databricks.sdk import WorkspaceClient
from databricks.sdk.service.serving import EndpointCoreConfigInput, ServedModelInput

serving_endpoint_name = f"dbdemos_endpoint_{catalog}_{db}"[:63]
latest_model_version = get_latest_model_version(model_name)

w = WorkspaceClient()
endpoint_config = EndpointCoreConfigInput(
    name=serving_endpoint_name,
    served_models=[
        ServedModelInput(
            model_name=model_name,
            model_version=latest_model_version,
            workload_size="Small",
            scale_to_zero_enabled=True,
            environment_vars={
                "DATABRICKS_TOKEN": "{{secrets/dbdemos/rag_sp_token}}",  # <scope>/<secret> that contains an access token
            }
        )
    ]
)

existing_endpoint = next(
    (e for e in w.serving_endpoints.list() if e.name == serving_endpoint_name), None
)
if existing_endpoint == None:
    print(f"Creating the endpoint {serving_endpoint_name}, this will take a few minutes to package and deploy the endpoint...")
    w.serving_endpoints.create_and_wait(name=serving_endpoint_name, config=endpoint_config)
else:
    print(f"Updating the endpoint {serving_endpoint_name} to version {latest_model_version}, this will take a few minutes to package and deploy the endpoint...")
    w.serving_endpoints.update_config_and_wait(served_models=endpoint_config.served_models, name=serving_endpoint_name)

Our endpoint is now deployed! You can search endpoint name on the [Serving Endpoint UI](#/mlflow/endpoints) and visualize its performance!

Let's run a REST query to try it in Python. As you can see, we send the `test sentence` doc and it returns an embedding representing our document.

In [0]:
question = "How can I track billing usage on my workspaces?"

answer = w.serving_endpoints.query(serving_endpoint_name, inputs=[{"query": question}])
print(answer.predictions[0])


### Let's give it a try, using Gradio as UI!

All you now have to do is deploy your chatbot UI. Here is a simple example using Gradio ([License](https://github.com/gradio-app/gradio/blob/main/LICENSE)). Explore the chatbot gradio [implementation](https://huggingface.co/spaces/databricks-demos/chatbot/blob/main/app.py).

*Note: this UI is hosted and maintained by Databricks for demo purpose and don't use the model you just created. We'll soon show you how to do that with Lakehouse Apps!*

In [0]:
display_gradio_app("databricks-demos-chatbot")

## Congratulations! You have deployed your first GenAI RAG model!

You're now ready to deploy the same logic for your internal knowledge base leveraging Lakehouse AI.

We've seen how the Lakehouse AI is uniquely positioned to help you solve your GenAI challenge:

- Simplify Data Ingestion and preparation with Databricks Engineering Capabilities
- Accelerate Vector Search  deployment with fully managed indexes
- Leverage Databricks LLama 2 foundation model endpoint
- Deploy realtime model endpoint to perform RAG and provide Q&A capabilities

Lakehouse AI is uniquely positioned to accelerate your GenAI deployment.

# Cleanup

To free up resources, please delete uncomment and run the below cell.

In [0]:
# /!\ THIS WILL DROP YOUR DEMO SCHEMA ENTIRELY /!\ 
# cleanup_demo(catalog, db, serving_endpoint_name, f"{catalog}.{db}.databricks_documentation_vs_index")